In [1]:
from utz import *
from njsp import CommitCrashes, ROOT_DIR
from njsp.cli.slack.channel_client import ChannelClient
from njsp.crash_log import get_crashes_df, DEFAULT_ROOT_SHA
from njsp.paths import CRASHES_RELPATH
from njsp.commit_crashes import load_pqt
chdir(ROOT_DIR)

In [2]:
%%time
# sha = 'f19266ca'
# sha = 'a61dbdbe'
# sha = '5dcc129d'
# sha = '38be41e0'  # 2023-02-01
# sha = 'd4b534bc'  # 2023-07-01
# sha = 'f6ee436b'  # 2024-02-08
# df = pd.read_parquet(f'crash_logs/{sha}.pqt')
df = pd.read_parquet('s3://data/njsp/log.pqt')
#df = get_crashes_df(head=sha, load_pqt=False)
df

CPU times: user 29.9 ms, sys: 15.1 ms, total: 45 ms
Wall time: 44.5 ms


,,rundate,kind,CCODE,CNAME,MCODE,MNAME,STREET,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,dt
accid,sha,,,,,,,,,,,,,,,,
10626,3590e7d3,2022-11-16 10:00:11-05:00,add,20,Union,2019,Union Twsp,None,78,Interstate 78 MP 50.8,1.0,1.0,0.0,0.0,0.0,0.0,2021-01-01 09:06:00
10627,3590e7d3,2022-11-16 10:00:11-05:00,add,11,Mercer,1113,West Windsor Twp,None,1,State Highway 1 MP 11.83,1.0,1.0,0.0,0.0,0.0,1.0,2021-01-02 09:07:00
10628,3590e7d3,2022-11-16 10:00:11-05:00,add,08,Gloucester,0805,Franklin Twsp,None,555,County 555 MP 24.9,1.0,0.0,1.0,0.0,0.0,2.0,2021-01-03 04:19:00
10630,3590e7d3,2022-11-16 10:00:11-05:00,add,15,Ocean,1533,Barnegat Twsp,None,444,Garden State Parkway MP 68.1,1.0,1.0,0.0,0.0,0.0,0.0,2021-01-01 07:45:00
10632,3590e7d3,2022-11-16 10:00:11-05:00,add,12,Middlesex,1225,Woodbridge Twsp,None,616,County 616 MP .29,1.0,0.0,0.0,1.0,0.0,2.0,2021-01-04 13:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12983,eab253d2,2024-02-06 10:00:03-05:00,update,02,Bergen,0212,East Rutherford Bor,Hackensack St,None,Hackensack St,1.0,0.0,0.0,1.0,0.0,NaN,2024-01-30 20:41:00
12984,eab253d2,2024-02-06 10:00:03-05:00,add,21,Warren,2108,Hackettstown Town,None,604,County 604 MP 1.2,1.0,0.0,0.0,0.0,1.0,0.0,2024-02-05 06:04:00
12985,4127ccf9,2024-02-07 10:00:02-05:00,add,14,Morris,1423,Morris Plains Boro,None,202,State Highway 202,1.0,1.0,0.0,0.0,0.0,NaN,2024-02-06 09:48:00


In [3]:
idx_keys = [ 'rundate', 'sha', 'kind', ]
def diff_obj(s):
    cur = s.cur
    prv = s.prv
    obj = { k: cur[k] for k in idx_keys }    
    if cur.kind in ['add', 'del']:
        return obj
    diffs = {}
    for k, c in cur.items():
        if k in idx_keys:
            continue
        p = prv[k]
        if p != c and not (isna(p) and isna(c)) and not (p is None and c is None):
            diffs[k] = p, c
    if not diffs:
        raise RuntimeError(f"Update with no diffs? {cur}, {prv}")
    return dict(**obj, diffs=diffs)

In [4]:
def diff_rows(d):
    r = d.reset_index()
    cur = r.copy() #.iloc[1:]
    cur.columns = pd.MultiIndex.from_tuples([ ('cur', col) for col in cur.columns ])
    prv = r.shift(1) #.dropna(how='all')
    prv.columns = pd.MultiIndex.from_tuples([ ('prv', col) for col in prv.columns ])
    ss = sxs(cur, prv).sort_index(axis=1)
    return DF(ss.apply(diff_obj, axis=1).tolist())

In [5]:
%%time
diffs_df = (
    df
    .groupby('accid', level=0)
    .apply(diff_rows)
    .reset_index(level=1, drop=True)
    .set_index('sha', append=True)
)
diffs_df

CPU times: user 4.21 s, sys: 10.3 ms, total: 4.22 s
Wall time: 4.22 s


,,rundate,kind,diffs
accid,sha,,,
10626,3590e7d3,2022-11-16 10:00:11-05:00,add,NaN
10627,3590e7d3,2022-11-16 10:00:11-05:00,add,NaN
10628,3590e7d3,2022-11-16 10:00:11-05:00,add,NaN
10630,3590e7d3,2022-11-16 10:00:11-05:00,add,NaN
10632,3590e7d3,2022-11-16 10:00:11-05:00,add,NaN
...,...,...,...,...
12983,eab253d2,2024-02-06 10:00:03-05:00,update,"{'LOCATION': ('HACKENSACK STREET', 'Hackensack..."
12984,eab253d2,2024-02-06 10:00:03-05:00,add,NaN
12985,4127ccf9,2024-02-07 10:00:02-05:00,add,NaN


In [6]:
diffs_col = diffs_df.diffs.dropna()
diffs_col

accid  sha     
10885  f19266ca    {'LOCATION': ('Birch Rodge Rd', 'Birch Ridge R...
11357  f19266ca    {'LOCATION': ('New Road', 'New Rd'), 'STREET':...
11453  f19266ca    {'HIGHWAY': (None, '705'), 'LOCATION': ('Valle...
11460  d7a3006d    {'INJURIES': (nan, 1.0), 'LOCATION': ('State H...
11461  d7a3006d    {'INJURIES': (nan, 0.0), 'LOCATION': ('County ...
                                         ...                        
12974  6ad2b9cf           {'LOCATION': ('130', 'State Highway 130')}
12978  984d92c8    {'HIGHWAY': (None, '130'), 'LOCATION': ('US 13...
12981  eab253d2    {'LOCATION': ('Sherman Avenue / Palmer Avenue'...
12982  eab253d2    {'LOCATION': ('EAST LINDEN AVE', 'East Linden ...
12983  eab253d2    {'LOCATION': ('HACKENSACK STREET', 'Hackensack...
Name: diffs, Length: 1326, dtype: object

In [7]:
def spread_tuple(r):
    cur, prv = r
    return Series(dict(prv=prv, cur=cur))

long_diffs = (
    diffs_col
    .apply(Series)
    .melt(
        var_name='var',
        value_name='val',
        ignore_index=False,
    )
    .dropna()
    .set_index('var', append=True)
    ['val']
    .apply(spread_tuple)
    .sort_index()
)
long_diffs

prv                             cur
accid sha      var                                                      
10885 f19266ca LOCATION   Birch Ridge Rd                  Birch Rodge Rd
               STREET     Birch Ridge Rd                  Birch Rodge Rd
11357 f19266ca LOCATION           New Rd                        New Road
               STREET             New Rd                        New Road
11453 f19266ca HIGHWAY               705                            None
...                                  ...                             ...
12981 eab253d2 STREET        Sherman Ave  Sherman Avenue / Palmer Avenue
12982 eab253d2 LOCATION  East Linden Ave                 EAST LINDEN AVE
               STREET    East Linden Ave                 EAST LINDEN AVE
12983 eab253d2 LOCATION    Hackensack St               HACKENSACK STREET
               STREET      Hackensack St               HACKENSACK STREET

[2266 rows x 2 columns]

Update-type histogram:

## Add/Update/Delete summary stats

In [8]:
df.kind.value_counts()

kind
add       2030
update    1326
del         78
Name: count, dtype: int64

In [9]:
hists = df.reset_index(level=1).groupby(lambda x:x).apply(lambda df: ''.join(df.kind.str[0]))
hists.value_counts().sort_index()

a         1037
ad          47
ada          4
adau         6
au         577
aud         14
auda         2
audad        1
audau        2
auu        263
auuu        53
auuud        1
auuuu        6
auuuuu       2
Name: count, dtype: int64

In [10]:
hists[hists.str.contains('aa')]

Series([], dtype: object)

In [11]:
hists[hists.str.contains('dd')]

Series([], dtype: object)

## Crashes with more than one entry

In [12]:
accid_entries = df.groupby(lambda x:x, level=0).size()
df.loc[accid_entries[accid_entries > 1].index]

rundate    kind CCODE      CNAME MCODE  \
accid sha                                                                 
10885 3590e7d3 2022-11-16 10:00:11-05:00     add    21     Warren  2109   
      f19266ca 2022-11-19 10:00:06-05:00  update    21     Warren  2109   
11357 3590e7d3 2022-11-16 10:00:11-05:00     add    12  Middlesex  1221   
      f19266ca 2022-11-19 10:00:06-05:00  update    12  Middlesex  1221   
11453 3590e7d3 2022-11-16 10:00:11-05:00     add    16    Passaic  1614   
...                                  ...     ...   ...        ...   ...   
12981 eab253d2 2024-02-06 10:00:03-05:00  update    02     Bergen  0260   
12982 984d92c8 2024-02-03 10:00:02-05:00     add    09     Hudson  0906   
      eab253d2 2024-02-06 10:00:03-05:00  update    09     Hudson  0906   
12983 001bb4ee 2024-02-05 10:00:03-05:00     add    02     Bergen  0212   
      eab253d2 2024-02-06 10:00:03-05:00  update    02     Bergen  0212   

                              MNAME             STREET HIGHWAY  \
accid sha                                                        
10885 3590e7d3        Hardwick Twsp     Birch Rodge Rd    None   
      f19266ca        Hardwick Twsp     Birch Ridge Rd    None   
11357 3590e7d3  South Brunswick Tws           New Road    None   
      f19266ca  South Brunswick Tws             New Rd    None   
11453 3590e7d3           Wayne Twsp        Valley Road    None   
...                             ...                ...     ...   
12981 eab253d2         Teaneck Twsp        Sherman Ave    None   
12982 984d92c8          Jersey City    EAST LINDEN AVE    None   
      eab253d2          Jersey City    East Linden Ave    None   
12983 001bb4ee  East Rutherford Bor  HACKENSACK STREET    None   
      eab253d2  East Rutherford Bor      Hackensack St    None   

                         LOCATION  FATALITIES  FATAL_D  FATAL_P  FATAL_T  \
accid sha                                                                  
10885 3590e7d3     Birch Rodge Rd         1.0      1.0      0.0      0.0   
      f19266ca     Birch Ridge Rd         1.0      1.0      0.0      0.0   
11357 3590e7d3           New Road         1.0      0.0      0.0      1.0   
      f19266ca             New Rd         1.0      0.0      0.0      1.0   
11453 3590e7d3        Valley Road         1.0      1.0      0.0      0.0   
...                           ...         ...      ...      ...      ...   
12981 eab253d2        Sherman Ave         1.0      1.0      0.0      0.0   
12982 984d92c8    EAST LINDEN AVE         1.0      0.0      1.0      0.0   
      eab253d2    East Linden Ave         1.0      0.0      1.0      0.0   
12983 001bb4ee  HACKENSACK STREET         1.0      0.0      0.0      1.0   
      eab253d2      Hackensack St         1.0      0.0      0.0      1.0   

                FATAL_B  INJURIES                  dt  
accid sha                                              
10885 3590e7d3      0.0       0.0 2021-05-16 08:05:00  
      f19266ca      0.0       0.0 2021-05-16 08:05:00  
11357 3590e7d3      0.0       0.0 2021-11-08 08:11:00  
      f19266ca      0.0       0.0 2021-11-08 08:11:00  
11453 3590e7d3      0.0       0.0 2021-12-27 20:11:00  
...                 ...       ...                 ...  
12981 eab253d2      0.0       NaN 2024-01-25 07:53:00  
12982 984d92c8      0.0       NaN 2024-02-03 01:06:00  
      eab253d2      0.0       NaN 2024-02-03 01:06:00  
12983 001bb4ee      0.0       NaN 2024-01-30 20:41:00  
      eab253d2      0.0       NaN 2024-01-30 20:41:00  

[2397 rows x 16 columns]

## Crashes with at least one deletion

In [13]:
del_ids = df[df.kind == 'del'].index.get_level_values(0)
deld = df.loc[del_ids]
deld

rundate    kind CCODE     CNAME MCODE  \
accid sha                                                                
11466 3590e7d3 2022-11-16 10:00:11-05:00     add    15     Ocean  1514   
      05740641 2022-12-14 10:00:13-05:00     del  None      None  None   
      dfb70741 2022-12-17 10:00:07-05:00     add    15     Ocean  1514   
      9580b28b 2022-12-18 10:00:05-05:00  update    15     Ocean  1514   
11552 3590e7d3 2022-11-16 10:00:11-05:00     add    13  Monmouth  1326   
...                                  ...     ...   ...       ...   ...   
12925 7812219f 2024-01-06 10:14:26-05:00     add    01  Atlantic  0104   
      cb237f2a 2024-01-08 10:00:03-05:00  update    01  Atlantic  0104   
      1d912f87 2024-01-09 10:00:02-05:00     del  None      None  None   
12943 ee515b87 2024-01-14 10:31:52-05:00     add    01  Atlantic  0111   
      ffeed969 2024-01-16 10:00:02-05:00     del  None      None  None   

                         MNAME                              STREET HIGHWAY  \
accid sha                                                                    
11466 3590e7d3   Lakewood Twsp                                None      88   
      05740641            None                                None    None   
      dfb70741   Lakewood Twsp                                None    None   
      9580b28b   Lakewood Twsp                                None      88   
11552 3590e7d3  Manalapan Twsp                        Pegasus Blvd    None   
...                        ...                                 ...     ...   
12925 7812219f      Buena Boro                                None     619   
      cb237f2a      Buena Boro                                None     619   
      1d912f87            None                                None    None   
12943 ee515b87   Galloway Twsp  61 w jimmie leeds road parking lot    None   
      ffeed969            None                                None    None   

                                          LOCATION  FATALITIES  FATAL_D  \
accid sha                                                                 
11466 3590e7d3                    State Highway 88         1.0      0.0   
      05740641                                None         NaN      NaN   
      dfb70741                                None         1.0      0.0   
      9580b28b             State Highway 88 MP .21         1.0      0.0   
11552 3590e7d3                        Pegasus Blvd         2.0      1.0   
...                                            ...         ...      ...   
12925 7812219f                                 619         1.0      1.0   
      cb237f2a                          County 619         1.0      1.0   
      1d912f87                                None         NaN      NaN   
12943 ee515b87  61 w jimmie leeds road parking lot         1.0      0.0   
      ffeed969                                None         NaN      NaN   

                FATAL_P  FATAL_T  FATAL_B  INJURIES                  dt  
accid sha                                                                
11466 3590e7d3      0.0      1.0      0.0       0.0 2022-01-03 21:46:00  
      05740641      NaN      NaN      NaN       NaN                 NaT  
      dfb70741      0.0      1.0      0.0       0.0 2022-01-03 21:46:00  
      9580b28b      0.0      1.0      0.0       0.0 2022-01-03 21:46:00  
11552 3590e7d3      1.0      0.0      0.0       0.0 2022-02-09 12:36:00  
...                 ...      ...      ...       ...                 ...  
12925 7812219f      0.0      0.0      0.0       NaN 2023-10-13 11:09:00  
      cb237f2a      0.0      0.0      0.0       NaN 2023-10-13 11:09:00  
      1d912f87      NaN      NaN      NaN       NaN                 NaT  
12943 ee515b87      0.0      1.0      0.0       NaN 2024-01-13 14:01:00  
      ffeed969      NaN      NaN      NaN       NaN                 NaT  

[205 rows x 16 columns]